# Sensitivity Analysis Notebook Ordinal vs Categorical

## Introduction

In this notebook, we perform a sensitivity analysis to compare the main model (ordinal) with an alternate (categorical) model.

The goal is to see if, in the future, the model will be able to handle non-ordinal comparisons such as food items.


## Libraries Used

First, we import the necessary libraries for data manipulation, probabilistic programming, and visualization.


In [1]:
# Data manipulation and analysis
import pandas as pd  # For data manipulation and analysis
import numpy as np  # For numerical operations and array manipulation

# Probabilistic programming and Bayesian statistical modeling
import pymc as pm  
import arviz as az  

# Data visualization
import matplotlib.pyplot as plt 
import seaborn as sns  

# Suppressing warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

import logging
logger = logging.getLogger("pymc")
logger.propagate = False
logger.setLevel(logging.ERROR)

## Load Preprocessed Data

### Load Cleaned Data

In [2]:
db_mon = pd.read_csv('data/mon_clean.csv')
db_med = pd.read_csv('data/med_clean.csv')

db_mon_online = pd.read_csv('data_online/mon_clean.csv')
db_med_online = pd.read_csv('data_online/med_clean.csv')

print('Participants in-person: ', len(db_mon['sub'].unique()), '\nparticipants online: ', len(db_mon_online['sub'].unique()))

Participants in-person:  66 
participants online:  332


### Extract Indices and Subject Numbers

In [3]:
# Assign a unique serial number for each participant.
db_mon['subn'] = db_mon['sub'].rank(method='dense').astype(int) - 1
db_mon_online['subn'] = db_mon_online['sub'].rank(method='dense').astype(int) - 1


# Count the number of unique subjects in the 'db_mon' dataset.
n_subs = db_mon['subn'].unique().shape[0]
n_subs_online = db_mon_online['subn'].unique().shape[0]

# Create a list of subject indices for all rows in the 'db_mon' dataset.
sub_idx = db_mon.subn.tolist()
sub_idx_online = db_mon_online.subn.tolist()

# Assign a unique serial number for each participant. This will be useful for indexing operations.
db_med['subn'] = db_med['sub'].rank(method='dense').astype(int) - 1
db_med_online['subn'] = db_med_online['sub'].rank(method='dense').astype(int) - 1


# Count the number of unique subjects in the 'db_med' dataset.
n_subs_med = db_med['subn'].unique().shape[0]
n_subs_med_online = db_med_online['subn'].unique().shape[0]

# Create a list of subject indices for all rows in the 'db_med' dataset.
sub_idx_med = db_med.subn.tolist()
sub_idx_med_online = db_med_online.subn.tolist()

## 1. Estimated Value Model (Ordinal)

In this model, we assume that the data is ordinal. While we still include the ambiguity attitude parameter, the risk attitude parameter becomes irrelevant because risk is value-based. This model is capable of estimating quantitative values for qualitative outcomes.

### Model Parameters:

- **Ambiguity Attitude (β):**
  - β = 0: Ambiguity neutral
  - β > 0: Ambiguity averse
  - β < 0: Ambiguity seeking

### Hyper Priors:
- **Ambiguity Aversion (β):** 0.65 (based on Levy et al., 2010)


We will estimate the quantitative values corresponding to these ordinal outcomes.


In [4]:
def estimate_value_ord(df, n_subs, idx):
    """
    Estimate the value of different reward levels using ordinal constraints and a common hyperprior for each level. 
    The model ensures that the levels are positive (ordinal constraints).

    Parameters:
    - df: DataFrame with trial-specific details, such as choices, value levels, risk, and ambiguity levels.
    - n_sub: Total number of subjects in the dataset.
    - idx: A list indicating the subject ID for each observation/trial.

    Returns:
    - trace: Samples from the posterior distribution of the model.
    """
    
    with pm.Model() as estimate:

        # Hyperparameters for group-level distributions
        bMu  = pm.Normal('bMu', .65, 1)     # Mean for ambiguity effect distribution
        bSig = pm.Gamma('bSig', 2, 1)       # SD for ambiguity effect distribution


       # Hyperparameters for group-level subjective value levels
        l1Mu = pm.TruncatedNormal('l1Mu', 4, 2, lower=0)  # Mean for value of level 1
        l2Mu = pm.TruncatedNormal('l2Mu', 4, 2, lower=0)  # ... level 2
        l3Mu = pm.TruncatedNormal('l3Mu', 4, 2, lower=0)  # ... level 3
        l4Mu = pm.TruncatedNormal('l4Mu', 4, 2, lower=0)  # ... level 4
        
        l1sd = pm.Gamma('l1sd', 3, 1)  # SD for value of level 1
        l2sd = pm.Gamma('l2sd', 3, 1)  # ... level 2
        l3sd = pm.Gamma('l3sd', 3, 1)  # ... level 3
        l4sd = pm.Gamma('l4sd', 3, 1)  # ... level 4
        
        # Subject-specific priors 
        β = pm.Normal('β',    bMu, bSig, shape = n_subs)   # Modulation of ambiguity effect
        γ = pm.Lognormal('γ', 0, 0.25, shape = n_subs)   # Inverse temperature, impacting choice stochasticity

        # Priors for subjective values of the different reward levels for each subject.
        level1 = pm.TruncatedNormal('level1', l1Mu, l1sd, lower = 0, shape = n_subs)
        level2 = pm.TruncatedNormal('level2', l2Mu, l2sd, lower = 0, shape = n_subs)
        level3 = pm.TruncatedNormal('level3', l3Mu, l3sd, lower = 0, shape = n_subs)
        level4 = pm.TruncatedNormal('level4', l4Mu, l4sd, lower = 0, shape = n_subs)

        # Calculate the total expected value for each trial by combining values from different levels
        val = (df['l1'].values * level1[idx] + 
               df['l2'].values * level2[idx] + 
               df['l3'].values * level3[idx] + 
               df['l4'].values * level4[idx]) 

        # Calculate adjusted probability by considering both risk and ambiguity levels modulated by β
        prob = (df['risk'].values) - (β[idx] * (df['ambiguity'].values/2))  

        # Compute the subjective value of the lottery option
        svLotto = val * prob
        svRef   = level1[idx]  # The subjective value of the reference option

        # Transform the SV difference between lottery and reference into a choice probability using the logistic function
        p  = (svLotto - svRef) / γ[idx]
        mu = pm.invlogit(p)

        # Likelihood of the observed choices given the computed probabilities
        choice = pm.Binomial('choice', 1, mu, observed=df['choice'])

        trace = pm.sample(idata_kwargs={'log_likelihood':True})
        
    return(trace)

## 2. Estimated Value Model (Semi-Categorical)

In this model, we assume that the data is categorical and can hold negative values, however, each category adds value to the previous category. 

In [5]:
def estimate_value(df, n_subs, idx):
    """
    Estimate the value of different reward levels using ordinal constraints and a common hyperprior for each level. 
    The model ensures that the levels are positive (ordinal constraints).

    Parameters:
    - df: DataFrame with trial-specific details, such as choices, value levels, risk, and ambiguity levels.
    - n_sub: Total number of subjects in the dataset.
    - idx: A list indicating the subject ID for each observation/trial.

    Returns:
    - trace: Samples from the posterior distribution of the model.
    """
    
    with pm.Model() as estimate:

        # Hyperparameters for group-level distributions
        bMu  = pm.Normal('bMu', .65, 1)     # Mean for ambiguity effect distribution
        bSig = pm.Gamma('bSig', 2, 1)       # SD for ambiguity effect distribution


       # Hyperparameters for group-level subjective value levels
        l1Mu = pm.Normal('l1Mu', 4, 2)  # Mean for value of level 1
        l2Mu = pm.Normal('l2Mu', 4, 2)  # ... level 2
        l3Mu = pm.Normal('l3Mu', 4, 2)  # ... level 3
        l4Mu = pm.Normal('l4Mu', 4, 2)  # ... level 4
        
        l1sd = pm.Gamma('l1sd', 3, 1)  # SD for value of level 1
        l2sd = pm.Gamma('l2sd', 3, 1)  # ... level 2
        l3sd = pm.Gamma('l3sd', 3, 1)  # ... level 3
        l4sd = pm.Gamma('l4sd', 3, 1)  # ... level 4
        
        # Subject-specific priors 
        β = pm.Normal('β',    bMu, bSig, shape = n_subs)   # Modulation of ambiguity effect
        γ = pm.Lognormal('γ', 0, 0.25, shape = n_subs)   # Inverse temperature, impacting choice stochasticity

        # Priors for subjective values of the different reward levels for each subject.
        level1 = pm.Normal('level1', l1Mu, l1sd, shape = n_subs)
        level2 = pm.Normal('level2', l2Mu, l2sd, shape = n_subs)
        level3 = pm.Normal('level3', l3Mu, l3sd, shape = n_subs)
        level4 = pm.Normal('level4', l4Mu, l4sd, shape = n_subs)

        # Calculate the total expected value for each trial by combining values from different levels
        val = (df['l1'].values * level1[idx] + 
               df['l2'].values * level2[idx] + 
               df['l3'].values * level3[idx] + 
               df['l4'].values * level4[idx]) 

        # Calculate adjusted probability by considering both risk and ambiguity levels modulated by β
        prob = (df['risk'].values) - (β[idx] * (df['ambiguity'].values/2))  

        # Compute the subjective value of the lottery option
        svLotto = val * prob
        svRef   = level1[idx]  # The subjective value of the reference option

        # Transform the SV difference between lottery and reference into a choice probability using the logistic function
        p  = (svLotto - svRef) / γ[idx]
        mu = pm.invlogit(p)

        # Likelihood of the observed choices given the computed probabilities
        choice = pm.Binomial('choice', 1, mu, observed=df['choice'])

        trace = pm.sample(idata_kwargs={'log_likelihood':True})
        
    return(trace)

## 3. Estimated Value Model (Full Categorical)

In this model, each category is calculated seperatly. This pose a problem in this datasets due to the increase in magnitude of the outcomes.

In [6]:
def estimate_value_cat(df, n_subs, idx):
    """
    Estimate the value of different reward levels using ordinal constraints and a common hyperprior for each level. 
    The model ensures that the levels are positive (ordinal constraints).

    Parameters:
    - df: DataFrame with trial-specific details, such as choices, value levels, risk, and ambiguity levels.
    - n_sub: Total number of subjects in the dataset.
    - idx: A list indicating the subject ID for each observation/trial.

    Returns:
    - trace: Samples from the posterior distribution of the model.
    """
    
    with pm.Model() as estimate:

        # Hyperparameters for group-level distributions
        bMu  = pm.Normal('bMu', .65, 1)     # Mean for ambiguity effect distribution
        bSig = pm.Gamma('bSig', 2, 1)       # SD for ambiguity effect distribution


       # Hyperparameters for group-level subjective value levels
        l1Mu = pm.Normal('l1Mu', 5, 2)  # Mean for value of level 1
        l2Mu = pm.Normal('l2Mu', 12, 2)  # ... level 2
        l3Mu = pm.Normal('l3Mu', 20, 2)  # ... level 3
        l4Mu = pm.Normal('l4Mu', 25, 2)  # ... level 4
        
        
        # Subject-specific priors 
        β = pm.Normal('β',    bMu, bSig, shape = n_subs)   # Modulation of ambiguity effect
        γ = pm.Lognormal('γ', 0, 0.25, shape = n_subs)   # Inverse temperature, impacting choice stochasticity

        # Priors for subjective values of the different reward levels for each subject.
        level1 = pm.Normal('level1', l1Mu, 2, shape = n_subs)
        level2 = pm.Normal('level2', l2Mu, 2, shape = n_subs)
        level3 = pm.Normal('level3', l3Mu, 2, shape = n_subs)
        level4 = pm.Normal('level4', l4Mu, 2, shape = n_subs)

        # Extract the 'level' column as a tensor
        level = df['level'].values  # Ensure this is a NumPy array or similar
    
        # Implement nested switch statements
        val = pm.math.switch(
            level == 1, level1[idx],
            pm.math.switch(
                level == 2, level2[idx],
                pm.math.switch(
                    level == 3, level3[idx],
                    level4[idx]  # Default case when level == 4
                )
            )
        ) 

        # Calculate adjusted probability by considering both risk and ambiguity levels modulated by β
        prob = (df['risk'].values) - (β[idx] * (df['ambiguity'].values/2))  

        # Compute the subjective value of the lottery option
        svLotto = val * prob
        svRef   = level1[idx]  # The subjective value of the reference option

        # Transform the SV difference between lottery and reference into a choice probability using the logistic function
        p  = (svLotto - svRef) / γ[idx]
        mu = pm.invlogit(p)

        # Likelihood of the observed choices given the computed probabilities
        choice = pm.Binomial('choice', 1, mu, observed=df['choice'])

        trace = pm.sample(idata_kwargs={'log_likelihood':True})
        
    return(trace)

## Run the models on the in-person datasets

In [7]:
mon_estimated_cat = estimate_value_cat(db_mon, n_subs, sub_idx)
med_estimated_cat = estimate_value_cat(db_med, n_subs_med, sub_idx_med)

mon_estimated = estimate_value(db_mon, n_subs, sub_idx)
med_estimated = estimate_value(db_med, n_subs_med, sub_idx_med)

mon_estimated_ord = estimate_value_ord(db_mon, n_subs, sub_idx)
med_estimated_ord = estimate_value_ord(db_med, n_subs_med, sub_idx_med)

## Run the models on the online datasets

In [8]:
mon_estimated_online = estimate_value(db_mon_online, n_subs_online, sub_idx_online)
med_estimated_online = estimate_value(db_med_online, n_subs_med_online, sub_idx_med_online)

mon_estimated_online_ord = estimate_value_ord(db_mon_online, n_subs_online, sub_idx_online)
med_estimated_online_ord = estimate_value_ord(db_med_online, n_subs_med_online, sub_idx_med_online)

mon_estimated_online_cat = estimate_value_cat(db_mon_online, n_subs_online, sub_idx_online)
med_estimated_online_cat = estimate_value_cat(db_med_online, n_subs_med_online, sub_idx_med_online)

## Compare models

In [9]:
print('Monetary in-Person')
compare_dict = {'Estimate Value':  mon_estimated,
                'Estimate Value Ordinal': mon_estimated_ord,
                'Estimate Value Categorical': mon_estimated_cat

}

comp = az.compare(compare_dict)
comp

Monetary in-Person


,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
Estimate Value Ordinal,0,-1560.883848,222.994940,0.000000,0.918119,45.454959,0.000000,True,log
Estimate Value,1,-1567.790372,240.239600,6.906524,0.081881,45.597207,4.098722,True,log
Estimate Value Categorical,2,-1610.418002,203.668597,49.534154,0.000000,44.976968,7.662955,True,log


In [10]:
print('Medical in-Person')
compare_dict = {'Estimate Value':  med_estimated,
                'Estimate Value Ordinal': med_estimated_ord,
                'Estimate Value Categorical': med_estimated_cat,

}

comp = az.compare(compare_dict)
comp

Medical in-Person


,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
Estimate Value Ordinal,0,-1411.570398,210.598234,0.000000,0.747705,45.084521,0.000000,True,log
Estimate Value,1,-1416.034498,226.218527,4.464100,0.252295,44.728629,4.348711,True,log
Estimate Value Categorical,2,-1439.435711,186.008677,27.865312,0.000000,43.691882,7.337146,True,log


In [11]:
print('Monetary on-line')
compare_dict = {'Estimate Value':  mon_estimated_online,
                'Estimate Value Ordinal': mon_estimated_online_ord,
                'Estimate Value Categorical': mon_estimated_online_cat

}

comp = az.compare(compare_dict)
comp

Monetary on-line


,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
Estimate Value Ordinal,0,-3790.653944,825.452374,0.000000,9.620189e-01,69.228866,0.000000,True,log
Estimate Value,1,-3818.563337,898.683684,27.909393,3.798114e-02,69.191796,7.713219,True,log
Estimate Value Categorical,2,-3950.989664,777.152465,160.335721,1.818301e-11,68.444847,12.877123,True,log


In [12]:
print('Medical On-Line')
compare_dict = {'Estimate Value':  med_estimated_online,
                'Estimate Value Ordinal': med_estimated_online_ord,
                'Estimate Value Categorical': med_estimated_online_cat


}

comp = az.compare(compare_dict)
comp

Medical On-Line


,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
Estimate Value Ordinal,0,-4103.178349,788.565283,0.000000,0.519858,72.849218,0.000000,True,log
Estimate Value,1,-4104.288061,868.755364,1.109712,0.480142,72.302638,10.146263,True,log
Estimate Value Categorical,2,-4302.661895,722.038121,199.483546,0.000000,68.868380,17.184448,True,log


The ordinal model fits the data best in the two tasks for both in-person and online datasets.</br>
Compared to the semi-categorical model, the model elpd_loo doesn't show a big difference, however, due to the lower complexity, it is the better model. 

## Extract Estimated Values

### Ordinal model

In [13]:
print("In-person: ")

print("level 1 (med): mean:", round(az.summary(med_estimated_ord, var_names=['level1'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated, var_names=['level1'])['sd'].mean(),2))
print("level 2 (med): mean:", round(az.summary(med_estimated_ord, var_names=['level2'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated, var_names=['level2'])['sd'].mean(),2))
print("level 3 (med): mean:", round(az.summary(med_estimated_ord, var_names=['level3'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated, var_names=['level3'])['sd'].mean(),2))
print("level 4 (med): mean:", round(az.summary(med_estimated_ord, var_names=['level4'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated, var_names=['level4'])['sd'].mean(),2))
print()
print("level 1 (mon): mean:", round(az.summary(mon_estimated_ord, var_names=['level1'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated, var_names=['level1'])['sd'].mean(),2))
print("level 2 (mon): mean:", round(az.summary(mon_estimated_ord, var_names=['level2'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated, var_names=['level2'])['sd'].mean(),2))
print("level 3 (mon): mean:", round(az.summary(mon_estimated_ord, var_names=['level3'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated, var_names=['level3'])['sd'].mean(),2))
print("level 4 (mon): mean:", round(az.summary(mon_estimated_ord, var_names=['level4'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated, var_names=['level4'])['sd'].mean(),2))

In-person: 
level 1 (med): mean: 6.92 SD:  1.64
level 2 (med): mean: 8.97 SD:  2.35
level 3 (med): mean: 7.01 SD:  2.93
level 4 (med): mean: 4.22 SD:  3.09

level 1 (mon): mean: 7.2 SD:  1.56
level 2 (mon): mean: 4.13 SD:  1.66
level 3 (mon): mean: 6.21 SD:  2.32
level 4 (mon): mean: 8.75 SD:  3.35


In [14]:
print("Online: ")

print("level 1 (med): mean:", round(az.summary(med_estimated_online_ord, var_names=['level1'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated_online, var_names=['level1'])['sd'].mean(),2))
print("level 2 (med): mean:", round(az.summary(med_estimated_online_ord, var_names=['level2'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated_online, var_names=['level2'])['sd'].mean(),2))
print("level 3 (med): mean:", round(az.summary(med_estimated_online_ord, var_names=['level3'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated_online, var_names=['level3'])['sd'].mean(),2))
print("level 4 (med): mean:", round(az.summary(med_estimated_online_ord, var_names=['level4'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated_online, var_names=['level4'])['sd'].mean(),2))
print()
print("level 1 (mon): mean:", round(az.summary(mon_estimated_online_ord, var_names=['level1'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated_online, var_names=['level1'])['sd'].mean(),2))
print("level 2 (mon): mean:", round(az.summary(mon_estimated_online_ord, var_names=['level2'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated_online, var_names=['level2'])['sd'].mean(),2))
print("level 3 (mon): mean:", round(az.summary(mon_estimated_online_ord, var_names=['level3'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated_online, var_names=['level3'])['sd'].mean(),2))
print("level 4 (mon): mean:", round(az.summary(mon_estimated_online_ord, var_names=['level4'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated_online, var_names=['level4'])['sd'].mean(),2))

Online: 
level 1 (med): mean: 8.62 SD:  2.3
level 2 (med): mean: 12.63 SD:  3.62
level 3 (med): mean: 4.66 SD:  2.91
level 4 (med): mean: 2.37 SD:  2.44

level 1 (mon): mean: 10.94 SD:  2.69
level 2 (mon): mean: 4.14 SD:  2.05
level 3 (mon): mean: 5.04 SD:  2.38
level 4 (mon): mean: 3.94 SD:  2.57


### Semi-Categorical

In [15]:
print("In-person: ")

print("level 1 (med): mean:", round(az.summary(med_estimated, var_names=['level1'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated, var_names=['level1'])['sd'].mean(),2))
print("level 2 (med): mean:", round(az.summary(med_estimated, var_names=['level2'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated, var_names=['level2'])['sd'].mean(),2))
print("level 3 (med): mean:", round(az.summary(med_estimated, var_names=['level3'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated, var_names=['level3'])['sd'].mean(),2))
print("level 4 (med): mean:", round(az.summary(med_estimated, var_names=['level4'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated, var_names=['level4'])['sd'].mean(),2))
print()
print("level 1 (mon): mean:", round(az.summary(mon_estimated, var_names=['level1'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated, var_names=['level1'])['sd'].mean(),2))
print("level 2 (mon): mean:", round(az.summary(mon_estimated, var_names=['level2'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated, var_names=['level2'])['sd'].mean(),2))
print("level 3 (mon): mean:", round(az.summary(mon_estimated, var_names=['level3'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated, var_names=['level3'])['sd'].mean(),2))
print("level 4 (mon): mean:", round(az.summary(mon_estimated, var_names=['level4'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated, var_names=['level4'])['sd'].mean(),2))

In-person: 
level 1 (med): mean: 6.83 SD:  1.64
level 2 (med): mean: 8.86 SD:  2.35
level 3 (med): mean: 6.93 SD:  2.93
level 4 (med): mean: 3.88 SD:  3.09

level 1 (mon): mean: 7.0 SD:  1.56
level 2 (mon): mean: 3.89 SD:  1.66
level 3 (mon): mean: 6.11 SD:  2.32
level 4 (mon): mean: 8.24 SD:  3.35


In [16]:
print("Online: ")

print("level 1 (med): mean:", round(az.summary(med_estimated_online, var_names=['level1'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated_online, var_names=['level1'])['sd'].mean(),2))
print("level 2 (med): mean:", round(az.summary(med_estimated_online, var_names=['level2'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated_online, var_names=['level2'])['sd'].mean(),2))
print("level 3 (med): mean:", round(az.summary(med_estimated_online, var_names=['level3'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated_online, var_names=['level3'])['sd'].mean(),2))
print("level 4 (med): mean:", round(az.summary(med_estimated_online, var_names=['level4'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated_online, var_names=['level4'])['sd'].mean(),2))
print()
print("level 1 (mon): mean:", round(az.summary(mon_estimated_online, var_names=['level1'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated_online, var_names=['level1'])['sd'].mean(),2))
print("level 2 (mon): mean:", round(az.summary(mon_estimated_online, var_names=['level2'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated_online, var_names=['level2'])['sd'].mean(),2))
print("level 3 (mon): mean:", round(az.summary(mon_estimated_online, var_names=['level3'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated_online, var_names=['level3'])['sd'].mean(),2))
print("level 4 (mon): mean:", round(az.summary(mon_estimated_online, var_names=['level4'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated_online, var_names=['level4'])['sd'].mean(),2))

Online: 
level 1 (med): mean: 8.49 SD:  2.3
level 2 (med): mean: 11.94 SD:  3.62
level 3 (med): mean: 4.5 SD:  2.91
level 4 (med): mean: 2.35 SD:  2.44

level 1 (mon): mean: 10.94 SD:  2.69
level 2 (mon): mean: 3.93 SD:  2.05
level 3 (mon): mean: 5.3 SD:  2.38
level 4 (mon): mean: 3.58 SD:  2.57


### Full categorical
This is the value of the category not added value

In [17]:
print("In-person: ")

print("level 1 (med): mean:", round(az.summary(med_estimated_cat, var_names=['level1'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated_cat, var_names=['level1'])['sd'].mean(),2))
print("level 2 (med): mean:", round(az.summary(med_estimated_cat, var_names=['level2'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated_cat, var_names=['level2'])['sd'].mean(),2))
print("level 3 (med): mean:", round(az.summary(med_estimated_cat, var_names=['level3'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated_cat, var_names=['level3'])['sd'].mean(),2))
print("level 4 (med): mean:", round(az.summary(med_estimated_cat, var_names=['level4'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated_cat, var_names=['level4'])['sd'].mean(),2))
print()
print("level 1 (mon): mean:", round(az.summary(mon_estimated_cat, var_names=['level1'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated_cat, var_names=['level1'])['sd'].mean(),2))
print("level 2 (mon): mean:", round(az.summary(mon_estimated_cat, var_names=['level2'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated_cat, var_names=['level2'])['sd'].mean(),2))
print("level 3 (mon): mean:", round(az.summary(mon_estimated_cat, var_names=['level3'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated_cat, var_names=['level3'])['sd'].mean(),2))
print("level 4 (mon): mean:", round(az.summary(mon_estimated_cat, var_names=['level4'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated_cat, var_names=['level4'])['sd'].mean(),2))

In-person: 
level 1 (med): mean: 5.88 SD:  0.69
level 2 (med): mean: 13.45 SD:  1.51
level 3 (med): mean: 19.11 SD:  1.76
level 4 (med): mean: 21.95 SD:  1.88

level 1 (mon): mean: 6.38 SD:  0.71
level 2 (mon): mean: 9.76 SD:  1.38
level 3 (mon): mean: 15.04 SD:  1.58
level 4 (mon): mean: 21.72 SD:  1.89


In [18]:
print("Online: ")

print("level 1 (med): mean:", round(az.summary(med_estimated_online_cat, var_names=['level1'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated_online_cat, var_names=['level1'])['sd'].mean(),2))
print("level 2 (med): mean:", round(az.summary(med_estimated_online_cat, var_names=['level2'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated_online_cat, var_names=['level2'])['sd'].mean(),2))
print("level 3 (med): mean:", round(az.summary(med_estimated_online_cat, var_names=['level3'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated_online_cat, var_names=['level3'])['sd'].mean(),2))
print("level 4 (med): mean:", round(az.summary(med_estimated_online_cat, var_names=['level4'])['mean'].mean(),2), "SD: ", round(az.summary(med_estimated_online_cat, var_names=['level4'])['sd'].mean(),2))
print()
print("level 1 (mon): mean:", round(az.summary(mon_estimated_online_cat, var_names=['level1'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated_online_cat, var_names=['level1'])['sd'].mean(),2))
print("level 2 (mon): mean:", round(az.summary(mon_estimated_online_cat, var_names=['level2'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated_online_cat, var_names=['level2'])['sd'].mean(),2))
print("level 3 (mon): mean:", round(az.summary(mon_estimated_online_cat, var_names=['level3'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated_online_cat, var_names=['level3'])['sd'].mean(),2))
print("level 4 (mon): mean:", round(az.summary(mon_estimated_online_cat, var_names=['level4'])['mean'].mean(),2), "SD: ", round(az.summary(mon_estimated_online_cat, var_names=['level4'])['sd'].mean(),2))

Online: 
level 1 (med): mean: 6.6 SD:  0.81
level 2 (med): mean: 15.16 SD:  1.65
level 3 (med): mean: 18.37 SD:  1.73
level 4 (med): mean: 20.07 SD:  1.78

level 1 (mon): mean: 9.02 SD:  0.91
level 2 (mon): mean: 12.03 SD:  1.61
level 3 (mon): mean: 16.65 SD:  1.65
level 4 (mon): mean: 19.51 SD:  1.73


In [19]:
%load_ext watermark
%watermark -n -u -v -iv -w -p xarray

Last updated: Fri Oct 25 2024

Python implementation: CPython
Python version       : 3.10.14
IPython version      : 8.26.0

xarray: 2024.7.0

arviz     : 0.17.1
pymc      : 4.1.7
logging   : 0.5.1.2
seaborn   : 0.12.2
numpy     : 1.23.5
matplotlib: 3.6.3
pandas    : 1.5.3

Watermark: 2.4.3

